In [9]:
import json
import random
import string
import time
import uuid

In [10]:
min_insurance_quote_duplicates = 0
max_insurance_quote_duplicates = 2

min_customer_decision_duplicates = 0
max_customer_decision_duplicates = 2

customers_number = 10
number_of_insurances = 200
insurance_quotes_number = 100

customer_id = "rgpp0wkpec"

# Insurance Quotes and Customer Decision Events

In [11]:
customer_ids = []
for i in range(customers_number):
    customer_ids += [''.join(random.choices(string.ascii_lowercase + string.digits, k=10))]

In [12]:
insurance_quote_ids = []
for i in range(insurance_quotes_number):
    insurance_quote_ids.append(i)

In [13]:
with open('insurance-quote.json', 'r') as file:
    insurance_quote_json = json.loads(file.read())

with open('insurance-quote-response.json', 'r') as file:
    insurance_quote_response_json = json.loads(file.read())

with open('customer-decision.json', 'r') as file:
    customer_decision_json = json.loads(file.read())
    
def jsons_with_id(insurance_quote, insurance_quote_response, customer_decision):
    insurance_quote_request_id = insurance_quote_ids[random.randint(0, insurance_quotes_number - 1)]
    new_insurance_quote_request = insurance_quote.copy()
    new_insurance_quote_response = insurance_quote_response.copy()
    new_customer_decision_request = customer_decision.copy()
    ts = int(time.time())
    
    new_insurance_quote_request['insuranceQuoteRequestDto']['id'] = insurance_quote_request_id
    new_insurance_quote_request['insuranceQuoteRequestDto']['customerInfo']['customerId'] = customer_id
    new_insurance_quote_request['date'] = ts

    new_insurance_quote_response['insuranceQuoteRequestId'] = insurance_quote_request_id
    new_insurance_quote_response['date'] = ts

    new_customer_decision_request['insuranceQuoteRequestId'] = insurance_quote_request_id
    new_customer_decision_request['date'] = ts

    return new_insurance_quote_request, new_insurance_quote_response, new_customer_decision_request

In [14]:
all_events = []
for i in range(number_of_insurances):
    insurance_quote, insurance_quote_response, customer_decision = jsons_with_id(insurance_quote_json, insurance_quote_response_json, customer_decision_json)
    insurance_quotes = [insurance_quote] *  random.randint(min_insurance_quote_duplicates, max_insurance_quote_duplicates)
    insurance_quote_responses = [insurance_quote_response] *  random.randint(min_insurance_quote_duplicates, max_insurance_quote_duplicates)
    customer_decisions = [customer_decision] * random.randint(min_customer_decision_duplicates, max_customer_decision_duplicates)
    all_events += list(map(lambda x: json.dumps(x), insurance_quotes)) + list(map(lambda x: json.dumps(x), insurance_quote_responses)) + list(map(lambda x: json.dumps(x), customer_decisions))

random.shuffle(all_events)

In [15]:
with open('insurance-quote-and-decision-events.txt', 'w') as file:
    for event in all_events:
        file.write(str(uuid.uuid4()))
        file.write(" ")
        event_json = json.loads(event)
        insurance_quote_id =  event_json['insuranceQuoteRequestId'] if 'insuranceQuoteRequestId' in event_json else event_json['insuranceQuoteRequestDto']['id'] 
        file.write(str(insurance_quote_id))
        file.write(" ")
        file.write(event)
        file.write('\n')

In [16]:
all_events[0]

'{"date": 1660693863, "insuranceQuoteRequestId": 23, "requestAccepted": true, "expirationDate": 1694663940000, "insurancePremium": {"amount": 1500, "currency": "CHF"}, "policyLimit": {"amount": 100, "currency": "CHF"}, "$type": "InsuranceQuoteResponseEvent"}'